Данный алгоритм рассматривает метод решения задачи о назначениях венгерским методом, при этом постановка задачи нечеткая.
Используемая литература:
1. https://studyres.com/doc/19607100/a-new-approach-for-solving-fuzzy-assignment-problem
2. https://neerc.ifmo.ru/wiki/index.php?title=Венгерский_алгоритм_решения_задачи_о_назначениях
3. https://www.ijitee.org/wp-content/uploads/papers/v9i1/A3912119119.pdf
4. https://www.ijcaonline.org/volume6/number4/pxc3871398.pdf (примеры: min 4, min 5, mod min 4)
5. https://math.semestr.ru/transp/t4.php

Мною далее рассматривается математическая постановка, описанная в работе [1].
Главным отличием от классической постановки задачи о назначениях является матрица С (например, матрица эффективности), составленная из нечетких чисел (при этом функции принадлежности в разных ячейках могут отличаться), а также дальнейшее сравнение нечетких чисел методом ранжирования Ягера, позволяющее приблизить результаты к анализу, более понятному человеческому глазу. Возможная интерпретация такой постановки матрицы эффективности: дан интервал (a, b, c, d), эффективность будет варьироваться от a до d для данного проекта заданным сотрудником, но вероятнее всего будет находиться от b до c.

Обработка данных в формате Excel файла.
Формат ввода данных:
1. Количество сотрудников/задач
2. Уточнение постановки задачи: минимизация (-1) или максимизация (1)
3. Матрица эффективности 

In [ ]:
import pandas as pd

def read_data_from_excel(file_path):
    df = pd.read_excel(file_path, header=None)

    n = int(df.iloc[0, 0])
    problem_flag = int(df.iloc[2, 0])
    c_matrix = df.iloc[4:4 + n, :n]
    c_matrix = c_matrix.applymap(lambda x: list(map(float, str(x).split()))).values.tolist()

    return n, problem_flag, c_matrix

Метод ранжирования Ягера.
Вход: текущая матрица.
Выход: агрегированное значение каждого нечеткого числа.

In [ ]:
from scipy.integrate import quad
import numpy as np

def yager_method(matrix):
    yager_matrix = np.zeros((len(matrix), len(matrix))).tolist() 
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            params = [each for each in matrix[i][j]]
            
            a, b, c, d = params[0], params[1], params[-2], params[-1]
            
            def lower_alpha(alpha):
                return a + alpha * (b - a)
            def upper_alpha(alpha):
                return d - alpha * (d - c)
            
            def integrand(alpha):
                return 0.5 * (lower_alpha(alpha) + upper_alpha(alpha))
            
            result, _ = quad(integrand, 0, 1)
            yager_matrix[i][j] = round(result, 1)

    return yager_matrix

Венгерский алгоритм:
1. Задача минимизации
   1. Преобразуем исходную матрицу с помощью метода ранжирования.
   2. Ищем построчно минимум и вычитаем из элементов в строке.
   3. Ищем по столбцам минимум и вычитаем из элементов в столбце.
   4. Проверка критерия оптимальности: попытка назначения. Для этого обходим по строкам и по столбцам и выбираем ноль (то есть назначаем сотрудника на задачу), если ноль в строки/столбце один. Нули же в соответствующем столбце/строке  помечаем как невыбранные и далее их не используем. Если получилось назначить всех, то нашли решение. Иначе переходим к E.
   5. Модификация матрицы: покрываем нули минимальным количеством линий. Среди незачеркнутых ячеек находим минимальное число, прибавляем его к элементам на пересечении линий и вычитаем из непокрытых элементов. Возвращаемся к D.

   Повторяем п.4 и п.5, пока не придем к оптимальному решению.
   Выводим результат назначения и целевую функцию.
   
3. Задача максимизации.
   Преобразуем исходную матрицу с помощью метода ранжирования.
   Ищем построчно максимальный элемент и вычитаем из элементов в строке.
   Умножаем матрицу на -1.
   Далее действия, аналогичные задаче минимизации.

In [ ]:
import numpy as np

def prep_for_hungarian(matrix):
    transformed_matrix = [
        [abs(element - max(row)) for element in row]
        for row in matrix
    ]
    return transformed_matrix

In [ ]:
def substract_row_minimum(matrix):
    for i in range(len(matrix)):
        if min(matrix[i]) > 0:
                min_el =  min(matrix[i])
                for j in range(len(matrix)):
                    matrix[i][j] = matrix[i][j] - min_el
                
    return matrix

In [ ]:
import numpy as np

def substract_col_minimum(matrix):
    matrix = np.array(matrix).T 
    for i in range(len(matrix)):
        if min(matrix[i]) > 0:
            min_el = min(matrix[i])
            for j in range(len(matrix[i])):
                matrix[i][j] = matrix[i][j] - min_el
                
    return matrix.T.tolist()


In [ ]:
import copy

def appointment_attempt(matrix):
    n = len(matrix)
    epsilon = 1e-9

    matrix_copy = copy.deepcopy(matrix)

    row_zeros = [sum(1 for j in range(n) if abs(matrix_copy[i][j]) < epsilon) for i in range(n)]
    col_zeros = [sum(1 for i in range(n) if abs(matrix_copy[i][j]) < epsilon) for j in range(n)]

    assign_vector = [-1] * n

    while not (all(x in [0, 1] for x in row_zeros) and all(x in [0, 1] for x in col_zeros)):
        for i in range(n):
            if row_zeros[i] == 1:
                col_index = next(j for j in range(n) if abs(matrix_copy[i][j]) < epsilon)

                for k in range(n):
                    if abs(matrix_copy[k][col_index]) < epsilon and k != i:
                        matrix_copy[k][col_index] = -1  # Покрываем ноль
                        col_zeros[col_index] -= 1
                        row_zeros[k] -= 1
                row_zeros[i] = 0
                assign_vector[i] = col_index

        for j in range(n):
            if col_zeros[j] == 1:
                row_index = next(i for i in range(n) if abs(matrix_copy[i][j]) < epsilon)

                for k in range(n):
                    if abs(matrix_copy[row_index][k]) < epsilon and k != j:
                        matrix_copy[row_index][k] = -1
                        row_zeros[row_index] -= 1
                        col_zeros[k] -= 1
                col_zeros[j] = 0
                assign_vector[row_index] = j

    return assign_vector


In [ ]:
def count_zeros(matrix, epsilon=1e-9):
    n = len(matrix)

    row_zeros = [sum(1 for j in range(n) if abs(matrix[i][j]) < epsilon) for i in range(n)]
    col_zeros = [sum(1 for i in range(n) if abs(matrix[i][j]) < epsilon) for j in range(n)]

    return row_zeros, col_zeros


In [ ]:
def calculate_new_matrix(matrix):
    n = len(matrix)

    new_matrix = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            row_zeros = sum(1 for k in range(n) if abs(matrix[i][k]) < 1e-9)
            col_zeros = sum(1 for k in range(n) if abs(matrix[k][j]) < 1e-9)
            
            if abs(matrix[i][j]) < 1e-9:
                new_matrix[i][j] = row_zeros + col_zeros - 1
            else:
                new_matrix[i][j] = row_zeros + col_zeros

    return new_matrix


In [ ]:
import copy

def modify_matrix(matrix):
    n = len(matrix)

    matrix_copy = copy.deepcopy(matrix)

    row_strike = [0] * len(matrix)
    col_strike = [0] * len(matrix)

    new_matrix = calculate_new_matrix(matrix_copy)

    while sum(sum(row) for row in new_matrix) > 0:
        max_element = max(max(value for value in row) for row in new_matrix)
        if max_element == 0:
            break
        else:
            
            for i, row in enumerate(new_matrix):
                for j, value in enumerate(row):
                    if value >= max_element:
                        max_element = value
                        max_index = (i, j)

            row_zeros, col_zeros = count_zeros(matrix_copy)
            if row_zeros[max_index[0]] >= col_zeros[max_index[1]]:
                for j in range(n):
                    matrix_copy[max_index[0]][j] =  -1

                row_strike[max_index[0]] = 1
            else:
                for i in range(n):
                    matrix_copy[i][max_index[1]] =  -1

                col_strike[max_index[1]] = 1
    
            new_matrix = calculate_new_matrix(matrix_copy)
    

    uncovered_elements = []
    for i in range(len(matrix_copy)):
        for j in range(len(matrix_copy)):
            if row_strike[i] == 0 and col_strike[j] == 0:
                uncovered_elements.append(matrix_copy[i][j])


    if not uncovered_elements:
        return matrix
    else:
        min_uncovered = min(uncovered_elements)
    
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                if row_strike[i] == 0 and col_strike[j] == 0:
                    matrix[i][j] -= min_uncovered
                elif row_strike[i] == 1 and col_strike[j] == 1:
                    matrix[i][j] += min_uncovered
    
    
        return matrix


In [ ]:
def print_matrix(matrix):
    num_rows = len(matrix)
    num_cols = len(matrix[0]) if matrix else 0

    max_width = max(
        len(f"{cell:.1f}") if isinstance(cell, (int, float)) else len(", ".join(f"{num:.1f}" for num in cell))
        for row in matrix for cell in row
        if isinstance(cell, (int, float)) or isinstance(cell, list)
    )
    max_width = max(max_width, len("Сотрудник №"))

    header = " " * (max_width + 2) + " | ".join(f"Задача №{j + 1}".center(max_width) for j in range(num_cols))
    print(header)
    print("-" * (len(header)))

    for i, row in enumerate(matrix):
        formatted_row = [
            "[" + ", ".join(f"{num:.1f}" for num in cell).center(max_width) + "]" if isinstance(cell, list)
            else f"{cell:.1f}".center(max_width)
            for cell in row
        ]
        row_label = f"Сотрудник №{i + 1}".ljust(max_width + 2)
        print(row_label + " | ".join(formatted_row))


In [ ]:
def print_assign(assign_vector):
    for num, index in enumerate(assign_vector):
        if num == -1:
            print(f'На задачу №{index + 1} никто не назначен')
        else:
            print((f'Cотрудник №{num + 1} назначен на задачу  №{index + 1}'))

In [ ]:
def hungarian_method(matrix, flag):

    if flag == 1:
        matrix = prep_for_hungarian(matrix)

    matrix = substract_row_minimum(matrix)
    matrix = substract_col_minimum(matrix)
    
    print('Приведенная матрица (в каждой строке и в каждом столбце должен стоять хотя бы 1 ноль): ')
    print_matrix(matrix)

    print('Попытка назначения для приведенной матрицы:')
    assign_vector = appointment_attempt(matrix)
    print_assign(assign_vector)
    print()
    
    while sum(assign_vector) < (n - 1)*n/2:
        matrix = modify_matrix(matrix)
        print()
        print('Модифицированная матрица:')
        print_matrix(matrix)
        assign_vector = appointment_attempt(matrix)
        print('Попытка назначения для модифицированной матрицы:')
        print_assign(assign_vector)
        print()

    
    return assign_vector, matrix

Начало алгоритма.

In [ ]:
import copy

if __name__ == "__main__":

    file_path = r"C:\Users\arina\OneDrive\Рабочий стол\диплом\HungarianFuzzy\max 3.xlsx"

    n, problem_flag, c_matrix = read_data_from_excel(file_path)

    c_matrix_orig = copy.deepcopy(c_matrix)
    print('Исходная матрица')
    print_matrix(c_matrix_orig)
    print()
    
    c_matrix = yager_method(c_matrix)
    c_matrix_yager = copy.deepcopy(c_matrix)
    print('Матрица Ягера')
    print_matrix(c_matrix_yager)
    print()
    
    assignment, optimal_matrix = hungarian_method(c_matrix, problem_flag)
    
    print("Конечная матрица после модификаций:")
    print_matrix(optimal_matrix)
    print()
    
    if problem_flag == 1:
        print('Для задачи о максимизации выгоды от назначений =>')
    else:
        print('Для задачи о минимизации затрат от назначений =>')

    print(' Порядок назначения:')
    for num, index in enumerate(assignment):
        print(f'  Номер сотрудника {num + 1} - Номер задания {index + 1}')
            

    result = 0

    
    for i in range(len(assignment)):
        result += c_matrix_yager[i][assignment[i]]

    print(f' Результат назначения {result}')